# Palmyra Fin

## Overview

[Palmyra Fin](https://writer.com/llms/palmyra-fin/) is Writer’s specialized language model for the finance industry, designed to support critical financial workflows with precision in terminology, document analysis, and complex investment analysis. Equipped with advanced capabilities for market analysis, risk assessment, and regulatory reporting, Palmyra Fin empowers financial organizations to streamline processes and make data-driven decisions confidently.

This cookbook demonstrates how to use Palmyra Fin with the [text generation](https://dev.writer.com/api-guides/api-reference/completion-api/text-generation) and [chat completions](https://dev.writer.com/api-guides/api-reference/completion-api/chat-completion) APIs.

## Prerequisites

- An [AI Studio](https://aap.writer.com) account. Sign up for a [free account here](https://aap.writer.com/register).
- A Writer API key. See how to [generate an API key here](https://dev.writer.com/api-guides/quickstart).

## Setup

First, install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()

## Usage

### Market and financial analysis

Palmyra Fin interprets and analyzes data trends, enabling financial analysts to generate insights and forecasts for strategic decision-making. This example demonstrates how to use Palmyra Fin to generate a stock forecast using the [text generation API](https://dev.writer.com/api-guides/text-generation). It streams the response to the console.

In [ ]:
prompt = """
Here is a stock data in csv format:
<csv>
Date,Open,High,Low,Close,Adjusted Close,Volume
2023-01-03,100.00,102.50,98.75,101.25,101.25,5000000
2023-01-04,101.50,103.00,100.25,102.00,102.00,4500000
2023-01-05,102.25,104.50,101.75,103.25,103.25,5500000
2023-01-06,103.00,104.75,102.50,104.00,104.00,6000000
2023-01-09,104.25,106.00,103.50,105.25,105.25,5800000
2023-01-10,105.00,107.50,104.25,106.75,106.75,7000000
2023-01-11,106.50,108.00,105.00,107.25,107.25,6500000
2023-01-12,107.00,109.25,106.50,108.75,108.75,8000000
2023-01-13,108.50,111.00,107.25,110.00,110.00,7500000
2023-01-16,110.25,112.50,109.00,111.25,111.25,9000000
2023-01-17,111.50,113.00,110.50,112.50,112.50,8500000
2023-01-18,112.00,114.25,111.00,113.75,113.75,10000000
2023-01-19,113.50,115.00,112.25,114.00,114.00,9500000
2023-01-20,114.25,116.50,113.00,115.25,115.25,8500000
2023-01-23,115.00,117.00,114.25,116.25,116.25,7500000
2023-01-24,116.50,118.75,115.50,118.00,118.00,6500000
2023-01-25,117.75,120.00,116.25,119.50,119.50,5500000
2023-01-26,119.00,121.25,117.75,120.75,120.75,5000000
2023-01-27,120.50,122.50,119.00,122.00,122.00,4500000
2023-01-30,122.25,124.50,120.50,123.75,123.75,4000000
2023-01-31,123.50,125.00,122.25,124.50,124.50,3500000
</csv>
Analyze the trends in the provided data, make predictions about future prices for this symbol, and create an investment plan based on your findings.
"""

response = client.completions.create(model="palmyra-fin", prompt=prompt, stream=True)
for chunk in response:
    print(chunk.value, end="", flush=True)

### Risk assessment

With a deep understanding of financial language and terminology, Palmyra Fin helps assess risk factors, generating precise reports that aid in risk management compliance. This next example demonstrates how to use Palmyra Fin to generate a risk assessment report using the text generation API.

In [ ]:
prompt = """
'Success Financial Services' needs to assess the credit risk of a newly acquired portfolio of corporate loans.
The portfolio includes loans to companies in different industries such metallurgy, machine-tool construction, instrumentation and light industry.
Additionally, the bank must ensure compliance with Basel III and other relevant regulations.
Can you generate a comprehensive risk assessment report that includes credit risk analysis for each loan, a summary of compliance with regulatory requirements, and recommendations for mitigating identified risks?
"""

response = client.completions.create(model="palmyra-fin", prompt=prompt, stream=True)
for chunk in response:
    print(chunk.value, end="", flush=True)

### Use Palmyra Fin in a chat

The following example demonstrates how to use Palmyra Fin in a chat with a general Palmyra LLM. It uses the [chat completions](https://dev.writer.com/api-guides/model-delegation) API and passes the `palmyra-fin` model as a tool that the LLM can use when a user asks for financial analysis.

First, define the tool that will use Palmyra Fin:

In [ ]:
tools = [{
    "type": "llm",
    "function": {
        "description": "A function that will invoke the llm identified by the given model, specialized in financial analysis. Any user request asking for financial advice or analysis should use this tool.",
        "model": "palmyra-fin"
    }
}]

Then, pass the `tools` array when creating a chat completion. The example below creates a hypothetical chat with a user who asks for financial analysis for a new product. It streams the response and also displays the value from the `llm_data` field of the response, which shows the tool call used to generate the response.

In [ ]:
messages = [{"role": "user", "content": "What is the risk of investing in the stock market?"}]

response = client.chat.chat(
    model="palmyra-x-004",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    stream=True  # The tools array defined earlier.
)

llm_data = None
for chunk in response:
    if chunk.choices[0].delta.llm_data is not None:
        llm_data = chunk.choices[0].delta.llm_data
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\nLLM data:", llm_data)
